# Segmenting and Clustering Neighborhoods in Toronto


In [4]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    folium-0.11.0              |             py_0          61 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    branca:          0.4.1-py_0        conda-forge
    folium:          

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Scrape Wikipedia data

In [6]:
import urllib.request

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [11]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"652df336-7a6c-4f10-8854-52123f8436fb","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [9]:
soup.title
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [10]:
all_tables=soup.find_all("table")
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B


In [12]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [13]:
A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [15]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C


['Not assigned\n',
 'Not assigned\n',
 'Parkwoods\n',
 'Victoria Village\n',
 'Regent Park, Harbourfront\n',
 'Lawrence Manor, Lawrence Heights\n',
 "Queen's Park, Ontario Provincial Government\n",
 'Not assigned\n',
 'Islington Avenue, Humber Valley Village\n',
 'Malvern, Rouge\n',
 'Not assigned\n',
 'Don Mills\n',
 'Parkview Hill, Woodbine Gardens\n',
 'Garden District, Ryerson\n',
 'Glencairn\n',
 'Not assigned\n',
 'Not assigned\n',
 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n',
 'Rouge Hill, Port Union, Highland Creek\n',
 'Not assigned\n',
 'Don Mills\n',
 'Woodbine Heights\n',
 'St. James Town\n',
 'Humewood-Cedarvale\n',
 'Not assigned\n',
 'Not assigned\n',
 'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood\n',
 'Guildwood, Morningside, West Hill\n',
 'Not assigned\n',
 'Not assigned\n',
 'The Beaches\n',
 'Berczy Park\n',
 'Caledonia-Fairbanks\n',
 'Not assigned\n',
 'Not assigned\n',
 'Not assigned\n',
 'Woburn\n',
 'Not assigned\n

In [20]:
df['Postal Code']=df['Postal Code'].str.strip()

In [21]:
df1=df[df['Borough'] != 'Not assigned\n']
df1

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [2]:
df1[df1['Neighborhood'] == 'Not assigned\n']


NameError: name 'df1' is not defined

In [22]:
df1.shape

(103, 3)

# ------------------------------------------------------------------------- Part 1 done

In [27]:
Geospatial=pd.read_csv('http://cocl.us/Geospatial_data')


In [24]:
Geospatial.head()
df2=pd.merge(df1,Geospatial,how='left',on=['Postal Code'])

In [30]:
df2.shape
df2['Borough']=df['Borough'].str.strip()
df2['Neighborhood']=df['Neighborhood'].str.strip()

In [31]:
df2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,Not assigned,Not assigned,43.753259,-79.329656
1,M4A,Not assigned,Not assigned,43.725882,-79.315572
2,M5A,North York,Parkwoods,43.654260,-79.360636
3,M6A,North York,Victoria Village,43.718518,-79.464763
4,M7A,Downtown Toronto,"Regent Park, Harbourfront",43.662301,-79.389494
5,M9A,North York,"Lawrence Manor, Lawrence Heights",43.667856,-79.532242
6,M1B,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.806686,-79.194353
7,M3B,Not assigned,Not assigned,43.745906,-79.352188
8,M4B,Etobicoke,"Islington Avenue, Humber Valley Village",43.706397,-79.309937
9,M5B,Scarborough,"Malvern, Rouge",43.657162,-79.378937


# ---------------------------------------------------------------------------------------Part 2 done

# Explore and cluster the neighborhoods in Downtown Toronto

In [33]:
toronto_data = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M7A,Downtown Toronto,"Regent Park, Harbourfront",43.662301,-79.389494
1,M1B,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.806686,-79.194353
2,M3C,Downtown Toronto,"Garden District, Ryerson",43.725900,-79.340923
3,M1G,Downtown Toronto,St. James Town,43.770992,-79.216917
4,M6H,Downtown Toronto,Berczy Park,43.669005,-79.442259


In [35]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [37]:
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto
    

In [38]:
CLIENT_ID = '41JQWV5PI44GEDBJGZOFXLKYHJ4SSGSDAQ20211TXAISKERQ' # your Foursquare ID
CLIENT_SECRET = '5XREPRA50HRCFQHMT3WAZZIFSQENXVXFSQHLA13XEOOXO1EA' # your Foursquare Secret
VERSION = '20200609' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 41JQWV5PI44GEDBJGZOFXLKYHJ4SSGSDAQ20211TXAISKERQ
CLIENT_SECRET:5XREPRA50HRCFQHMT3WAZZIFSQENXVXFSQHLA13XEOOXO1EA


In [42]:
toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [44]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_data.loc[0, 'Longitude']

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6623015, -79.3894938.


#### Now, let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

In [46]:
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=41JQWV5PI44GEDBJGZOFXLKYHJ4SSGSDAQ20211TXAISKERQ&client_secret=5XREPRA50HRCFQHMT3WAZZIFSQENXVXFSQHLA13XEOOXO1EA&v=20200609&ll=43.6623015,-79.3894938&radius=500&limit=100'

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edf8ca3fb34b5001b31dda4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 33,
  'suggestedBounds': {'ne': {'lat': 43.6668015045, 'lng': -79.38328496866619},
   'sw': {'lat': 43.657801495499996, 'lng': -79.3957026313338}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b9d206bf964a520e69136e3',
       'name': "Queen's Park",
       'location': {'address': 'University Ave.',
        'crossStreet': 'at Wellesley Ave.',
        'lat': 43.66394609897775,
        'lng': -79.39217952520835,
        'labeledL

In [48]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Sushi Box,Sushi Restaurant,43.662960,-79.386580
2,Mercatto,Italian Restaurant,43.660391,-79.387664
3,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870
4,T-Swirl Crepe,Creperie,43.663452,-79.384125


In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

33 venues were returned by Foursquare.


In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel


In [53]:
print(toronto_venues.shape)
toronto_venues.head()

(150, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.662301,-79.389494,Queen's Park,43.663946,-79.392180,Park
1,"Regent Park, Harbourfront",43.662301,-79.389494,Sushi Box,43.662960,-79.386580,Sushi Restaurant
2,"Regent Park, Harbourfront",43.662301,-79.389494,Mercatto,43.660391,-79.387664,Italian Restaurant
3,"Regent Park, Harbourfront",43.662301,-79.389494,Neo Coffee Bar,43.660140,-79.385870,Coffee Shop
4,"Regent Park, Harbourfront",43.662301,-79.389494,T-Swirl Crepe,43.663452,-79.384125,Creperie


In [54]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,15,15,15,15,15,15
Central Bay Street,4,4,4,4,4,4
Christie,43,43,43,43,43,43
"Commerce Court, Victoria Hotel",13,13,13,13,13,13
"Garden District, Ryerson",21,21,21,21,21,21
"Harbourfront East, Union Station, Toronto Islands",4,4,4,4,4,4
"Queen's Park, Ontario Provincial Government",2,2,2,2,2,2
"Regent Park, Harbourfront",32,32,32,32,32,32
"Richmond, Adelaide, King",5,5,5,5,5,5


In [55]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 83 uniques categories.


In [57]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burrito Place,Bus Stop,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,College Stadium,Construction & Landscaping,Cosmetics Shop,Creperie,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Print Shop,Pub,Restaurant,Sandwich Place,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Theater,Trail,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
toronto_onehot.shape

(150, 84)

In [59]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burrito Place,Bus Stop,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,College Stadium,Construction & Landscaping,Cosmetics Shop,Creperie,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Park,Pet Store,Pharmacy,Pizza Place,Print Shop,Pub,Restaurant,Sandwich Place,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Theater,Trail,Wings Joint,Yoga Studio
0,Berczy Park,0.00,0.000000,0.000000,0.00000,0.000000,0.133333,0.066667,0.066667,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.066667,0.066667,0.066667,0.066667,0.133333,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.066667,0.0000,0.00000,0.000000,0.00000,0.000000
1,Central Bay Street,0.25,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.25,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000,0.000000
2,Christie,0.00,0.023256,0.000000,0.00000,0.000000,0.023256,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.046512,0.000000,0.023256,0.023256,0.000000,0.000000,0.00,0.023256,0.023256,0.000000,0.000000,0.069767,0.00000,0.00000,0.00,0.0,0.023256,0.00000,0.000000,0.023256,0.000000,0.00000,0.000000,0.00000,0.00,0.0,0.000000,0.000000,0.023256,0.023256,0.046512,0.000000,0.00,0.186047,0.023256,0.000000,0.000000,0.00000,0.000000,0.046512,0.023256,0.000000,0.069767,0.023256,0.023256,0.00,0.023256,0.023256,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.0,0.023256,0.046512,0.000000,0.00,0.00000,0.00,0.023256,0.000000,0.000000,0.0000,0.00000,0.023256,0.00000,0.023256
3,"Commerce Court, Victoria Hotel",0.00,0.076923,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.153846,0.00000,0.00000,0.00,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.0,0.000000,0.076923,0.000000,0.000000,0.000000,0.076923,0.00,0.000000,0.000000,0.076923,0.000000,0.00000,0.153846,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.076923,0.00000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.076923,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000,0.000000
4,"Garden Dis

In [60]:
toronto_grouped.shape

(11, 84)

In [62]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
           venue  freq
0         Bakery  0.13
1       Pharmacy  0.13
2  Grocery Store  0.07
3    Supermarket  0.07
4    Music Venue  0.07


----Central Bay Street----
                venue  freq
0             Airport  0.25
1                Park  0.25
2    Business Service  0.25
3   Electronics Store  0.25
4  Italian Restaurant  0.00


----Christie----
                venue  freq
0    Greek Restaurant  0.19
1  Italian Restaurant  0.07
2         Coffee Shop  0.07
3           Bookstore  0.05
4          Restaurant  0.05


----Commerce Court, Victoria Hotel----
                       venue  freq
0                      Hotel  0.15
1                Coffee Shop  0.15
2  Middle Eastern Restaurant  0.08
3             Sandwich Place  0.08
4        American Restaurant  0.08


----Garden District, Ryerson----
              venue  freq
0       Coffee Shop   0.1
1  Asian Restaurant   0.1
2        Restaurant   0.1
3        Beer Store   0.1
4               Gym   0.1


----Harbourfro

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Pharmacy,Bakery,Bar,Park,Music Venue,Grocery Store,Middle Eastern Restaurant,Brewery,Bank,Pet Store
1,Central Bay Street,Airport,Business Service,Electronics Store,Park,Cosmetics Shop,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,College Stadium
2,Christie,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Ice Cream Shop,Furniture / Home Store,Restaurant,Yoga Studio,Frozen Yogurt Shop,Juice Bar
3,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Mediterranean Restaurant,Sandwich Place,Middle Eastern Restaurant,Intersection,Burrito Place,Gym,Gas Station,Fried Chicken Joint
4,"Garden District, Ryerson",Gym,Coffee Shop,Beer Store,Restaurant,Asian Restaurant,Bike Shop,Japanese Restaurant,Sandwich Place,Chinese Restaurant,Clothing Store


##  Cluster Neighborhoods

In [67]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 1, 1, 4, 2, 1, 1, 0], dtype=int32)

In [68]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M7A,Downtown Toronto,"Regent Park, Harbourfront",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Park,Diner,Fried Chicken Joint,Wings Joint,Creperie,Gym,College Cafeteria
1,M1B,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Yoga Studio,Creperie,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,College Stadium,Construction & Landscaping
2,M3C,Downtown Toronto,"Garden District, Ryerson",43.725900,-79.340923,1,Gym,Coffee Shop,Beer Store,Restaurant,Asian Restaurant,Bike Shop,Japanese Restaurant,Sandwich Place,Chinese Restaurant,Clothing Store
3,M1G,Downtown Toronto,St. James Town,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Soccer Field,Yoga Studio,Creperie,Clothing Store,College Auditorium,College Cafeteria,College Stadium,Construction & Landscaping
4,M6H,Downtown Toronto,Berczy Park,43.669005,-79.442259,1,Pharmacy,Bakery,Bar,Park,Music Venue,Grocery Store,Middle Eastern Restaurant,Brewery,Bank,Pet Store


In [69]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,0,Coffee Shop,Korean Restaurant,Soccer Field,Yoga Studio,Creperie,Clothing Store,College Auditorium,College Cafeteria,College Stadium,Construction & Landscaping


#### Cluster 2

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Park,Diner,Fried Chicken Joint,Wings Joint,Creperie,Gym,College Cafeteria
2,Downtown Toronto,1,Gym,Coffee Shop,Beer Store,Restaurant,Asian Restaurant,Bike Shop,Japanese Restaurant,Sandwich Place,Chinese Restaurant,Clothing Store
4,Downtown Toronto,1,Pharmacy,Bakery,Bar,Park,Music Venue,Grocery Store,Middle Eastern Restaurant,Brewery,Bank,Pet Store
6,Downtown Toronto,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Ice Cream Shop,Furniture / Home Store,Restaurant,Yoga Studio,Frozen Yogurt Shop,Juice Bar
7,Downtown Toronto,1,Trail,Park,Bakery,Basketball Court,Construction & Landscaping,Yoga Studio,Creperie,Clothing Store,Coffee Shop,College Auditorium
9,Downtown Toronto,1,Sandwich Place,Food & Drink Shop,Park,Department Store,Gym / Fitness Center,Breakfast Spot,Hotel,Cosmetics Shop,Coffee Shop,College Auditorium
10,Downtown Toronto,1,Coffee Shop,Hotel,Mediterranean Restaurant,Sandwich Place,Middle Eastern Restaurant,Intersection,Burrito Place,Gym,Gas Station,Fried Chicken Joint


#### Cluster 3

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,2,Fast Food Restaurant,Print Shop,Yoga Studio,Creperie,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,College Stadium,Construction & Landscaping


#### Cluster 4

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,3,Airport,Business Service,Electronics Store,Park,Cosmetics Shop,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,College Stadium


In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
#### Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]